# Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')
import missingno as msno
import numpy as np
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.impute import KNNImputer
from sklearn.neighbors import LocalOutlierFactor
from lightgbm import LGBMRegressor

In [ ]:
#Read the Data
df=pd.read_csv("/kaggle/input/hitters-baseball-data/Hitters.csv")

In [ ]:
data=df.copy()
df.head()

# DATA UNDERSTANDING

In [ ]:
def data_understanding(df):
    print('############shape##############')
    print(df.shape)
    print('############types##############')
    print(df.dtypes)
    print('############head###############')
    print(df.head())
    print('############info###############')
    print(df.info())
    print('############nunique###############')
    print(df.nunique())

In [ ]:
# There are 322 observations and int-float-object types of features in this data set
data_understanding(df)

In [ ]:
print("Num of Object Variables:", df.select_dtypes(object).shape[1])
print("Num of Integer Variables:", df.select_dtypes("integer").shape[1])
print("Num of Float Variables:", df.select_dtypes("float").shape[1])

In [ ]:
df["League"].value_counts()

In [ ]:
df["League"].value_counts().plot.barh()

In [ ]:
df["NewLeague"].value_counts()

In [ ]:
df["NewLeague"].value_counts().plot.barh()

In [ ]:
df["Division"].value_counts()

In [ ]:
df["Division"].value_counts().plot.barh()

In [ ]:
sns.distplot(df['Salary'])

In [ ]:
#If the missing values don't come from Salary(target feature), i would have thought to assign mean according to these results.
# Because, there seems to be a relation between categoric variables and Salary values for example there is an important differences between being E Division and W Devision.
print("New League= A" ,df[df["NewLeague"]=="A"].agg({"Salary":"mean"}))
print("New League= N" ,df[df["NewLeague"]=="N"].agg({"Salary":"mean"}))
print("League= A" ,df[df["League"]=="A"].agg({"Salary":"mean"}))
print("League= N" ,df[df["League"]=="N"].agg({"Salary":"mean"}))
print("Division= E" ,df[df["Division"]=="E"].agg({"Salary":"mean"}))
print("Division= W" ,df[df["Division"]=="W"].agg({"Salary":"mean"}))

# Data Preprocessing

In [ ]:
#There are 59 null values in Hitters data set
df.isnull().sum().sum()

In [ ]:
# All these NA values comes from "Salary" feature
df.isnull().sum()

In [ ]:
df[df.Salary.isnull()==True].head()

In [ ]:
msno.bar(df)

# Outliers

In [ ]:
#Statistical view for all features
df.describe().T

In [ ]:
# Descriptive Analysis
df.describe([0.05,0.25,0.50,0.75,0.95,0.99]).T

In [ ]:
sns.boxplot(x = df["Salary"])
plt.show()

In [ ]:
def outlier_thresholds(dataframe, col_name, q1=0.05, q3=0.95):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [ ]:
#When the quarters of 1% and quartiles of 99% were examined first, no outlier was found.
lower, upper=outlier_thresholds(df, 'Salary', q1=0.01, q3=0.99)
print(df[(df['Salary']<lower) | (df['Salary']>upper)].shape[0])

In [ ]:
#Then, when the quarters of 25% and quarters of 75% were examined, an outlier was found.
#Conclusion: Observation analysis against the dependent variable is applied according to quartiles of 25 and 75. 
#Business sector information may remain untouched.
lower, upper=outlier_thresholds(df, 'Salary', q1=0.25, q3=0.75)
print(df[(df['Salary']<lower) | (df['Salary']>upper)].shape[0])

In [ ]:
#Later, when quarters of 5% and quarters of 95% were examined, no outlier was found.
lower, upper=outlier_thresholds(df, 'Salary')
print(df[(df['Salary']<lower) | (df['Salary']>upper)].shape[0])

In [ ]:
# numerical variables
def numeric_cols(df):
    numeric_cols = [col for col in df.columns if df[col].dtypes != "O"]
    return numeric_cols

In [ ]:
#Here, how many outlier observations in all variables in quartiles of 25 and 75 are accessed.
for col in numeric_cols(df):
    lower, upper=outlier_thresholds(df, col, 0.25, 0.75)
    count=df[(df[col]<lower) | (df[col]>upper)].shape[0]
    if count!=0:
        print(col, 'yes')
        print(count)
    else:
        print(col, 'no')

In [ ]:
def replace_with_thresholds(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if low_limit > 0:
        dataframe.loc[(dataframe[col_name] < low_limit), col_name] = low_limit
        dataframe.loc[(dataframe[col_name] > up_limit), col_name] = up_limit
    else:
        dataframe.loc[(dataframe[col_name] > up_limit), col_name] = up_limit
        
    return dataframe

In [ ]:
df=replace_with_thresholds(df, 'Salary')

In [ ]:
sns.boxplot(df['Salary'])

In [ ]:
for i in numeric_cols(df):

    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 4))
    sns.histplot(df[i], bins=10, ax=axes[0])
    axes[0].set_title(i)
    
    sns.boxplot(df[i], ax=axes[1])
    axes[1].set_title(i)
   
    sns.kdeplot(df[i], ax=axes[2])
    axes[2].set_title(i)
    plt.show()

In [ ]:
# correlation analysis
df.corr()

In [ ]:
def correlation(df, size=[20, 15]):
    f, ax = plt.subplots(figsize= [20,15])
    sns.heatmap(df.corr(), annot=True, fmt=".2f", ax=ax, cmap = "magma" )
    ax.set_title("Correlation Matrix", fontsize=20)
    plt.show()

In [ ]:
correlation(df)

In [ ]:
# Correlation analysis of numerical variables was performed.
def find_corr(df, num_col_names, limit=0.55):
    high_corrs={}
    for col in num_col_names:
        if col=='Salary':
            pass
        else:
            corr=df[[col, 'Salary']].corr().loc[col, 'Salary']
            print(col, corr)
            if abs(corr)>limit:
                high_corrs[col]=corr
    return high_corrs

In [ ]:
high_corrs = find_corr(df, numeric_cols(df))

In [ ]:
#Two variables with high correlation.
print(high_corrs)

In [ ]:
sns.scatterplot(x= df['CRuns'], y=df.Salary)

In [ ]:
sns.scatterplot(x= df['CRBI'], y=df.Salary)

# Local Outlier Factor

In [ ]:
def lof_scores(df):
    clf=LocalOutlierFactor(n_neighbors=20, contamination=0.1)
    clf.fit_predict(df)
    df_scores=clf.negative_outlier_factor_
    sns.boxplot(df_scores)
    plt.show()
    return df_scores
    
def lof(df, df_scores, threshold):
    not_outlier = df_scores >threshold
    value= df[df_scores == threshold]
    outliers = df[~not_outlier] 
    res=outliers.to_records(index=False)
    res[:] = value.to_records(index = False)
    not_outlier_df = df[not_outlier]
    outliers = pd.DataFrame(res, index = df[~not_outlier].index)
    df_res = pd.concat([not_outlier_df, outliers], ignore_index = True)
    return df_res

# New variables

In [ ]:
#New variables were created with the most appropriate variables according to their proportions.
#The data set includes the data obtained by the players in 1986 and throughout their careers and how many years of experience they have. 
#We add the annual averages of these data and the ratio of the 1986 data to the overall performance.
def new_var(df):
    df['AtBat_new'] = df['AtBat'] / df['CAtBat']
    df['Hits_new'] = df['Hits'] / df['CHits']
    df['HmRun_new'] = (df['HmRun'] / df['CHmRun']).fillna(0)
    df['Runs_new'] = df['Runs'] / df['CRuns']
    df['RBI_new'] = (df['RBI'] / df['CRBI']).fillna(0)
    df['Walks_new'] = (df['Walks'] / df['CWalks']).fillna(0)

    df["CAtBat_rate"] = df["CAtBat"] / df["Years"]
    df["CHits_rate"] = df["CHits"] / df["Years"]
    df["CHmRun_rate"] = df["CHmRun"] / df["Years"]
    df["Cruns_rate"] = df["CRuns"] / df["Years"]
    df["CRBI_rate"] = df["CRBI"] / df["Years"]
    df["CWalks_rate"] = df["CWalks"] / df["Years"]
    
    return df

# Min-Max Scaler

In [ ]:
def minmax_scaler(dataframe, col_names, feature_range=(0,1)):
    minmax_scaler = MinMaxScaler(feature_range=feature_range)
    col_names=[col for col in col_names if col !="Salary"]
    dataframe[col_names] = minmax_scaler.fit_transform(dataframe[col_names])
    return dataframe

We will create different data sets for different scenarios that we will apply for salary estimation.

# First option

In [ ]:
#drop NA values
df1=df.dropna()
df1.shape

In [ ]:
df1=minmax_scaler(df1, numeric_cols(df1))

In [ ]:
df1.isnull().sum().sum()

In [ ]:
# Variables with 2 categories
def var_two_cat(df):    
    bins_cols=[col for col in df.columns if df[col].dtype=='O' and df[col].nunique()==2]
    return bins_cols

In [ ]:
print(var_two_cat(df1))

In [ ]:
def label_encoder(df, bins_cols):
    for col in bins_cols:
        le=LabelEncoder()
        df[col]=le.fit_transform(df[col])
    return df

In [ ]:
df1=label_encoder(df1, var_two_cat(df1))

In [ ]:
df1.name='df1'
df1.head()

# Second option

In [ ]:
#This is second option and method is fill NA values with mean
df2=df.copy()

In [ ]:
df2['New_Year'] = pd.cut(x=df2['Years'], bins=[0, 3, 6, 10, 15, 19, 24], ).astype("O")

df2['New_Year'].value_counts().plot.barh()

In [ ]:
df2.isnull().sum().sum()

In [ ]:
msno.bar(df2)

In [ ]:
df2['Salary']=df2['Salary'].fillna(df2.groupby(['New_Year', "League", 'Division'])['Salary'].transform('mean'))

In [ ]:
df2.isnull().sum().sum()

In [ ]:
df2.head()

In [ ]:
df2.drop('New_Year', axis=1, inplace=True)

In [ ]:
df2=minmax_scaler(df2, numeric_cols(df2))

In [ ]:
df2=label_encoder(df2, var_two_cat(df2))

In [ ]:
df2.head()

In [ ]:
df2.head()
df2.name='df2'

# Third option

In [ ]:
df3=df.copy()

In [ ]:
df3=minmax_scaler(df3, numeric_cols(df3))

In [ ]:
df3=label_encoder(df3, var_two_cat(df3))

In [ ]:
df3.head()

In [ ]:
# We fill in the missing observations with the KNN algorithm and create the dataset named 'df_knn_imp':
def knn_imputer(df, n):
    imputer = KNNImputer(n_neighbors = n)
    df_filled = imputer.fit_transform(df)
    df_knn_imp = pd.DataFrame(df_filled,columns = df.columns)
    return df_knn_imp

In [ ]:
df3=knn_imputer(df3, 4)

In [ ]:
df3.isnull().sum().sum()

In [ ]:
df3.name='df3'
df3.head()

# Fourth option

In [ ]:
#Filling Missing Data with KNN and Suppressing Outliers to create 'df4'
df4=df.copy()

In [ ]:
df4.head()

In [ ]:
df4=minmax_scaler(df4, numeric_cols(df4))

In [ ]:
df4=label_encoder(df4, var_two_cat(df4))

In [ ]:
df4.head()

In [ ]:
df4=knn_imputer(df4, 4)

In [ ]:
array=np.sort(lof_scores(df4))
print(array)

In [ ]:
sns.boxplot(array[array>array[16]])

In [ ]:
df_scores=lof_scores(df4)
df4=lof(df4, df_scores, np.sort(df_scores)[16])

In [ ]:
df4.isnull().sum().sum()

In [ ]:
df4.name='df4'
df4.head()

# Fifth option

In [ ]:
df5=df.copy()

In [ ]:
df5=new_var(df5)

In [ ]:
df5=label_encoder(df5, var_two_cat(df5))

In [ ]:
df5=knn_imputer(df5, 4)

In [ ]:
df_scores=lof_scores(df5)
print(np.sort(df_scores))

In [ ]:
df5=lof(df5, df_scores, np.sort(df_scores)[13])

In [ ]:
df5=minmax_scaler(df5, numeric_cols(df5))

In [ ]:
df5.isnull().sum().sum()

In [ ]:
df5.name='df5'
df5.head()

# Model

In [ ]:
def reg_model(df, Y, algo, test_size=0.20):
    X=df.drop(Y, axis=1)
    Y=df[[Y]]
    X_train, X_test, Y_train, Y_test=train_test_split(X, Y, test_size=test_size, random_state=42)
    model=algo.fit(X_train, Y_train)
    Y_train_pred=model.predict(X_train)
    train_rmse=np.sqrt(mean_squared_error(Y_train, Y_train_pred))
    print(df.name)
    print(type(model).__name__)
    print("Train RMSE: {}".format(train_rmse))
    
    Y_test_pred=model.predict(X_test)
    test_rmse=np.sqrt(mean_squared_error(Y_test, Y_test_pred))
    print("Test RMSE: {}".format(test_rmse))
    print('###################################')
    return (df.name, type(model).__name__, train_rmse, test_rmse)

In [ ]:
models=[LinearRegression(), Ridge(), Lasso(), ElasticNet(), LGBMRegressor()]
dataframes=[df1, df2, df3, df4, df5]
results={'frame':[], 'model':[], 'train_rmse':[], 'test_rmse':[]}

In [ ]:
for frame in dataframes:
    for model in models:
        res=reg_model(frame, 'Salary', model)
        results['frame'].append(res[0])
        results['model'].append(res[1])
        results['train_rmse'].append(res[2])
        results['test_rmse'].append(res[3])

In [ ]:
results=pd.DataFrame(results)
results

In [ ]:
for frame in dataframes:
    sns.barplot(x= 'test_rmse', y = 'model', data=results[results['frame']==frame.name])
    plt.title(frame.name)
    plt.xlabel('Errors')
    plt.ylabel('Models')
    plt.show()

# MODEL TUNING

In [ ]:
def model_tuning(df, Y, algo_cv, algo, grid, test_size=0.20, cv=10):
    X=df.drop(Y, axis=1)
    Y=df[[Y]]
    X_train, X_test, Y_train, Y_test=train_test_split(X, Y, random_state=42, test_size=test_size)
    if type(algo()).__name__=='LGBMRegressor':
        model=algo()
        model_cv=algo_cv(model, grid, cv=10, n_jobs=-1, verbose=2)
        model_cv.fit(X_train, Y_train)
        model_tuned=LGBMRegressor(learning_rate=model_cv.best_params_['learning_rate'],
                         max_depth=model_cv.best_params_['max_depth'],
                         n_estimators=model_cv.best_params_['n_estimators'],
                         boosting_type=model_cv.best_params_['boosting_type'],
                         colsample_bytree=model_cv.best_params_['colsample_bytree'])
    else:   
        model_cv=algo_cv(alphas=grid, cv=cv)
        model_cv.fit(X_train, Y_train)
        model_tuned=algo(alpha=model_cv.alpha_)
    model_tuned.fit(X_train, Y_train)
    print(df.name)
    print(type(model_tuned).__name__)
    Y_train_pred=model_tuned.predict(X_train)
    train_rmse=np.sqrt(mean_squared_error(Y_train, Y_train_pred))
    print("Train RMSE:{}".format(train_rmse))
    Y_test_pred=model_tuned.predict(X_test)
    test_rmse=np.sqrt(mean_squared_error(Y_test, Y_test_pred))
    print("Test RMSE:{}".format(test_rmse))
    print('#####################')
    return (df.name, type(model_tuned).__name__, train_rmse, test_rmse)

In [ ]:
models={Ridge: RidgeCV, Lasso:LassoCV, ElasticNet:ElasticNetCV, LGBMRegressor: GridSearchCV}
results_tuned={'frame':[], 'model':[], 'train_rmse':[], 'test_rmse':[]}
lgbm_grid={
    'colsample_bytree':[0.4, 0.5, 0.6, 0.9, 1],
    'boosting_type':['dart'],
    'learning_rate':[0.01, 0.1, 0.5, 0.2],
    'num_leaves':[30, 31, 32],
           'n_estimators':[20, 40, 100, 200, 150, 90, 110],
           'max_depth':[1, 2, 3, 4, 5, 6, 7, 8]}
alphas = [0.1,0.01, 0.005, 0.05, 0.001,0.2,0.3,0.5,0.8,0.9]

In [ ]:
for frame in dataframes:
    for model in models:
        if type(model()).__name__=='LGBMRegressor':
            res=model_tuning(frame, 'Salary', models[model], model, lgbm_grid)
        else:
            res=model_tuning(frame, 'Salary', models[model], model, alphas)
        results_tuned['frame'].append(res[0])
        results_tuned['model'].append(res[1])
        results_tuned['train_rmse'].append(res[2])
        results_tuned['test_rmse'].append(res[3])

In [ ]:
results_tuned=pd.DataFrame(results_tuned)
results_tuned

In [ ]:
for frame in dataframes:
    sns.barplot(x= 'test_rmse', y = 'model', data=results_tuned[results_tuned['frame']==frame.name])
    plt.title(frame.name)
    plt.xlabel('Errors')
    plt.ylabel('Models')
    plt.show()

# Reporting

# 1. Data Understanding

The aim of this study is to set up different models for the Hitters data set and minimize error scores in 4 data sets that have undergone different preprocessing.

The studies conducted are as follows:

1) Hitters Data Set was read.


2) With Exploratory Data Analysis:

Structural information of the dataset was checked.

Types of variables in data set were examined.

The size information of the data set has been accessed.

The number of missing observations from which variable in the data set was accessed. It was observed that there were 59 missing observations only in "Salary" which was dependent variable.


Descriptive statistics of the data set were examined.

# 2. Data Preprocessing

3) In Data PreProcessing:

For Df1: NaN values ​​deleted, Outliers detected by LOF and fall. Dummy variables are created. The x variables are normalized.

For Df2: the year column is categorized and the column "new_year" is created. NaN values ​​were filled by looking at the averages of "Salary" in age, league and department variables. Dummy variables were created. The x variables are normalized.

For Df3: NaN values ​​were filled with KNN. Dummy variables were created. The x variables are normalized.

For Df4: NaN values ​​were filled with KNN. Outliers detected on the LOF side were suppressed. Dummy variables are created. The x variables are normalized.

For Df5: New variables were added, NaN values ​​were filled with KNN, outliers detected on the LOF side were suppressed.

# 3. Modeling

4) During the Model Building phase:

Using the Linear, Ridge, Lasso, ElasticNet and Light GBM machine learning models, RMSE values representing the difference between actual values and predicted values were calculated. Later, hyperparameter optimizations were applied for Ridge, Lasso and ElasticNet to further reduce the error value.

The 5 data frames mentioned above were trained with models.(Linear Regression, Ridge, Lasso and Light GBM)

df1 -> the best rmse -> LightGBM 353.8

df2 -> the best rmse -> LightGBM 295.3

df3 -> the best rmse -> LightGBM 284.8

df4 -> the best rmse -> LightGBM 153.2

df5 -> the best rmse -> Lasso 212.8

# 4. Tuning

Trying different values ​​for hyper parameters, it was tried to find the best model.

Overall, lower error was obtained with Light GBM in 4 data frames. The best result was obtained with 141.1 with Light GBM trained with df4.

# 5. Conclusion

When the model created as a result of Light GBM Hyperparameter optimization was applied to the df4 Data Frame, the lowest RMSE was obtained. (141.1)